Projekt samodzielny

Mateusz Miotła

Prognoza 13.8

Model polega na analizie szeregów czasowych na bazie ARIMA. Analizuje on miesięczne wskaźniki inflacji od 1982 pobrane z bazy danych GUS.

In [1]:
#import bibliotek
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

#link do pobrania danych w formacie CSV
csv= 'https://stat.gov.pl/download/gfx/portalinformacyjny/pl/defaultstronaopisowa/4741/1/1/miesieczne_wskazniki_cen_towarow_i_uslug_konsumpcyjnych_od_1982_roku.csv'

# Wczytywanie danych bezpośrednio z pliku CSV do DataFrame
df = pd.read_csv(csv, delimiter=';', encoding='cp1250')


#oczyszczanie danych
df = df.drop(['Nazwa zmiennej', 'Jednostka terytorialna', 'Flaga', 'Unnamed: 7', 'Unnamed: 8'  ], axis=1)
df = df[df['Sposób prezentacji'] == 'Analogiczny miesiąc poprzedniego roku = 100']
df = df.drop('Sposób prezentacji', axis=1)
df['Wartość'] = df['Wartość'].str.replace(',', '.').astype(float)
df= df.dropna()


#tworzenie kolumny Dzień zawierającej ostatni dzień danego miesiąca
df['Dzień'] = pd.to_datetime(df['Miesiąc'].astype(str) + '-' + df['Rok'].astype(str), format='%m-%Y').dt.days_in_month

#tworzenie kolumny Data zawierającej pełną datę
df['Data'] = pd.to_datetime(df['Rok'].astype(str) + '-' + df['Miesiąc'].astype(str) + '-' + df['Dzień'].astype(str))

#ustawienie daty jako indeksu
df.set_index('Data', inplace=True)
#sortowanie danych według daty
df.sort_index(inplace=True)

#dalsze oczyszczanie danych
df.drop(['Rok', 'Miesiąc', 'Dzień'], axis=1, inplace=True)

#tworzenie modelu ARIMA
model = ARIMA(df['Wartość'], order=(1, 2, 1), freq='M')
model_fit = model.fit()

# Przewidywanie wartości na 31.05.2023
predicted_cpi = model_fit.forecast(steps=1)[0]
print('Przewidywana wartość na 31.05.2023:', round(predicted_cpi-100, 1))

Przewidywana wartość na 31.05.2023: 13.8


c:\Users\cl9ma\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
